# Sprint 1--Collect and Display Atomic Records

## Set Up Session

In [1]:
# Initialize PySpark
APP_NAME = "Collect and Display Atomic Records"

# If there is no SparkSession, create the environment
try:
  sc and spark
except NameError as e:
  import findspark
  findspark.init()
  import pyspark
  import pyspark.sql

  sc = pyspark.SparkContext()
  spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()

print("PySpark initiated.")

PySpark initiated.


## ETL

### Load Raw Dataset in CSV

In [2]:
# Load from storage:
on_time_dataframe = spark.read.format(
    'com.databricks.spark.csv'
).options(    
    header='true',    
    treatEmptyValuesAsNulls='true'
).load(
    '../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.csv'
)

on_time_dataframe.registerTempTable("on_time_performance")

In [3]:
# Extract only needed fields:
on_time_dataframe = spark.sql(
"""
    SELECT  
        Year, 
        Quarter, 
        Month, 
        DayofMonth, 
        DayOfWeek, 
        FlightDate,  
        Carrier, 
        TailNum, 
        FlightNum,  
        Origin, 
        OriginCityName, 
        OriginState,  
        Dest, 
        DestCityName, 
        DestState,  
        DepTime, 
        cast(DepDelay as float), 
        cast(DepDelayMinutes as int),  
        cast(TaxiOut as float), 
        cast(TaxiIn as float),  
        WheelsOff, 
        WheelsOn,  
        ArrTime, 
        cast(ArrDelay as float), 
        cast(ArrDelayMinutes as float),  
        cast(Cancelled as int), 
        cast(Diverted as int),  
        cast(ActualElapsedTime as float), 
        cast(AirTime as float),  
        cast(Flights as int), 
        cast(Distance as float),  
        cast(CarrierDelay as float), 
        cast(WeatherDelay as float),   
        cast(NASDelay as float),  
        cast(SecurityDelay as float),   
        cast(LateAircraftDelay as float),  
        CRSDepTime, 
        CRSArrTime
    FROM
        on_time_performance
"""
)

on_time_dataframe.registerTempTable("on_time_performance")
on_time_dataframe.count()

5819079

### Dump as Parquet & JSON Lines to Compress

#### Write

In [4]:
on_time_dataframe.toJSON().saveAsTextFile(
    "../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.jsonl.gz",    
    "org.apache.hadoop.io.compress.GzipCodec"    
)

on_time_dataframe.write.parquet(
    "../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.parquet"
)

#### Compare File Sizes in Different Formats

In [5]:
%%bash

# On time performance dataset in JSON Lines
ls -sh "../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.csv"

2.5G ../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.csv


In [6]:
%%bash

# On time performance dataset in JSON Lines
ls -sh "../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.jsonl.gz"

total 283M
   0 _SUCCESS
 15M part-00000.gz
 14M part-00001.gz
 15M part-00002.gz
 15M part-00003.gz
 15M part-00004.gz
 15M part-00005.gz
 14M part-00006.gz
 15M part-00007.gz
 16M part-00008.gz
 14M part-00009.gz
 15M part-00010.gz
 15M part-00011.gz
 15M part-00012.gz
 14M part-00013.gz
 14M part-00014.gz
 15M part-00015.gz
 15M part-00016.gz
 15M part-00017.gz
 14M part-00018.gz
8.7M part-00019.gz


In [7]:
%%bash

# On time performance dataset in JSON Lines
ls -sh "../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.parquet"

total 129M
   0 _SUCCESS
6.5M part-00000-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.7M part-00001-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.9M part-00002-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.8M part-00003-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.7M part-00004-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.5M part-00005-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.4M part-00006-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.6M part-00007-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.6M part-00008-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.7M part-00009-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.7M part-00010-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.4M part-00011-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.9M part-00012-6cf1f2dc-57cd-41da-b2c4-514368f98f18-c000.snappy.parquet
6.2M part-00013-6cf1f2dc-5

#### Read Back

In [8]:
on_time_dataframe = spark.read.json(
    "../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.jsonl.gz"
)

on_time_dataframe = spark.read.parquet(
    "../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.parquet"    
)

### Inject to MongoDB 

#### Set Up Session

In [9]:
import pymongo
import pymongo_spark

In [10]:
# Important: ACTIVATE Pymongo Spark
pymongo_spark.activate()

on_time_dataframe = spark.read.parquet(
    "../data/On_Time_On_Time_Performance_2015/On_Time_On_Time_Performance_2015.parquet"
)

# Row in RDD has to be converted to dict to avoid https://jira.mongodb.org/browse/HADOOP-276
on_time_dict = on_time_dataframe.rdd.map(
    lambda row: row.asDict()
)

on_time_dict.saveToMongoDB(
    'mongodb://localhost:27017/agile_data_science.on_time_performance'
)